# Entities as Experts

This notebook is a code implementation of the paper "Entities as Experts: Sparse Memory Access with Entity Supervision" by Févry, Baldini Soares, FitzGerald, Choi, Kwiatowski.

## Problem definition and high-level model description

We want to perform question answering on typical one-shot questions that require external knowledge or context. For example, in order to answer the question "Which country was Charles Darwin born in?" one needs some text providing answers on typical structured scenarios.

In this case, however, we want to rely on knowledge-graph extracted information. For example, in the question given here, we can prune out unrelated to the antropologist and evolution theorist Charles Darwins, e.g. Charles River, Darwin City etc. 

In the paper, the authors propose to augment BERT in the task of cloze-type question answering by leveraging an Entity Memory extracted from e.g. a Knoweldge Graph.

![Entity as Experts description](images/eae_highlevel.png)

The Entity Memory is a simple bunch of embeddings of entities extracted from a Knowledge Graph. Relationships are ignored (see the Facts as Experts paper and notebook to see how they could be used).

## Datasets

> We assume access to a corpus $D={(xi,mi)}$,where all entity mentions are detected but not necessarily  all  linked  to  entities.   We  use  English Wikipedia as our corpus, with a vocabulary of 1m entities. Entity links come from hyperlinks, leading to 32m 128 byte contexts containing 17m entity links.

In the appendix B, it is explained that:

> We build our training corpus of contexts paired with entity mention labels from the 2019-04-14 dump of English Wikipedia. We first divide each article into chunks of 500 bytes,resulting in a corpus of 32 million contexts withover 17 million entity mentions. We restrict our-selves  to  the  one  million  most  frequent  entities
(86% of the linked mentions).

Given that the dump 2019-04-14 is not available at the time of writing, we will adopt the revision 2020-11-01.

Entities are thus partially extracted by link annotations (e.g. they associate with each token a mention if that token belongs to a wikipedia url).

## Mention Detection

> In addition to the Wikipedia links, we annotaten each sentence with unlinked mention spans using the mention detector from Section 2.2

The mention detection head discussed in Section 2.2 is a simple BIO sequence: each token is annotated with a B (beginning), I (inside) or O (outside) if they are respectivelly beginning, inside or outside of a mention. The reason why we use both BIO and EL is to avoid inconsistencies.

There is a catch. In the paper, they explain they used Google NLP APIs to perform entity detection and linking on large-scale Wikipedia entries, that is, to have a properly annotated Wikipedia dataset.

Since we cannot technically afford this, we will use spacy's entity detection and linking capabilities as a baseline. Data quality 

## Chunking

- Split articles by chunks of 500 bytes (assuming unicode encoding).
- We will elide sentences till the last period to make sure they reach such limit without giving weird effects.

## Tokenization:

- BERT Tokenizer (e.g. Wordpiece) using lowercase vocabulary, limited to 128 distinct word-piece tokens.

In [97]:
from tools.providers import WikipediaProvider

#WikipediaProvider.dump_full_dataset(revision="20201101")
from trec_car import read_data
from tools.dumps import wrap_open
from collections import defaultdict
import spacy

from trec_car.read_data import Page, Section, List, Para, ParaLink, ParaText

nlp = spacy.load("en_core_web_md", pipeline=["tokenizer"])

In [113]:
def handle_section(skel, toks, links):
    for subskel in skel.children:
        visit_section(subskel, toks, links)


def handle_list(skel, toks, links):
    visit_section(body, toks, links)

def handle_para(skel: Para, toks, links):
    paragraph = skel.paragraph
    bodies = paragraph.bodies

    for body in bodies:
        visit_section(body, toks, links)

def handle_paratext(body: ParaBody, toks, links):
    #lemmas = [x.text for x in nlp(body.get_text())]
    #toks.extend(lemmas)
    #links.extend(["PAD"] * len(lemmas))
    pass

def handle_paralink(body: ParaLink, toks, links):
    #lemmas = [x.text for x in nlp(body.anchor_text)]
    #toks.extend(lemmas)
    #links.append([body.page] + ["PAD"] * (len(lemmas) - 1))
    pass

def nothing():
    return lambda body, toks, links: None

handler = defaultdict(nothing, {Section: handle_section,
                     Para: handle_para,
                     List: handle_list,
                     ParaLink: handle_paralink,
                     ParaText: handle_paratext})


def visit_section(skel, toks, links):
    
    # Recur on the sections
    handler[type(skel)](skel, toks, links)

In [114]:
def test():
    with wrap_open("wikipedia/car-wiki2020-01-01/enwiki2020.cbor", "rb") as toc:
        for idx, page in enumerate(read_data.iter_annotations(toc)):
            links = []
            toks = []
            for skel in page.skeleton:
                visit_section(skel, toks, links)

In [ ]:
%prun test()

In [ ]:
import torch
from torch.utils.data import Dataset, 
import tqdm
from transformers import BertTokenizer, BertConfig
from tools.dumps import get_filename_path

from keras.preprocessing.sequence import pad_sequences

import random

from trec_car.read_data import AnnotationsFile, ParagraphsFile

class WikipediaCBOR(Dataset):
    """
    This is a simple CBOR loader.
    """
    def __init__(self, cbor_path):
        """
        :param cbor_path the path of the wikipedia cbor export
        """
        # Let trec deal with that in my place
        
        self.cbor_path = get_filename_path(cbor_path)
        self.cbor_toc_annotations = AnnotationsFile(self.cbor_path)
        self.cbor_toc_paragraphs = ParagraphsFile(self.cbor_path)
        
        self.keys = selfself.cbor_toc_paragraphs.keys()
        
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
        
        # preprocess and find the top k unique wikipedia links
        
    
        
    def __len__(self):
        return len(self.keys)
    
    def __get_whole_text(paragraph):
        # Tokenize by word. When a link is detected, 
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx)
            idx = idx.tolist()
    
        paragraphs = [self.cbor_toc_paragraphs.get(self.keys[i]) for i in idx]
                
        # TODO: can we parallelize this?
        
        
    
    return torch.tensor(paragraphs)
    
        

In [93]:
with wrap_open("wikipedia/car-wiki2020-01-01/enwiki2020.cbor", "rb") as toc:
    #toc.seek(46102636493)
    
    for idx, page in enumerate(read_data.iter_pages(toc)):
        if idx >= 10:
            break
        print(page)
        page.

Page(Anarchism)
Page(Albedo)
Page(Autism)
Page(A)
Page(Achilles)
Page(Alabama)
Page(Abraham Lincoln)
Page(An American in Paris)
Page(Aristotle)
Page(Academy Award for Best Production Design)


## Model

In the paper, the authors explain they used a modified BERT.

In [12]:
from torch.nn import Module, Embedding, Dropout, ModuleList, Linear
import torch.nn as nn
import torch
import math

GELU = torch.nn.GELU
LayerNorm = torch.nn.LayerNorm

l0 = 4
l1 = 8

class SublayerConnection(Module):
    """A residual connection followed by layer norm and applied dropout.
    """
    
    def __init__(self, size: int, dropout: float):
        """
        :param size the size of the layer norm
        :param dropout the dropout rate
        """
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = Dropout(dropout)
    
    def forward(self, x, sublayer):
        return x + self.dropout(self.norm(sublayer(x)))

class PositionwiseFeedForward(Module):
    """
    Positionwise Feed Forward: a Feed-Forward for each token.
    We also add dropout here and use GELU(why?)
    """
    def __init__(self, d_hidden, embedding_size, dropout=0.1):
        """
        :param d_hidden the hidden depth of the FCN
        :param embedding_size the size of a single embedding to convert
        """
        self(PositionwiseFeedForward, self).__init__()
        self.w_1 = Linear(embedding_size, d_hidden)
        self.w_2 = Linear(d_hidden, embedding_size)
        self.dropout = Dropout(dropout)
        self.activation = GELU()
    
    def forward(self, x):
        return self.w_2(self.dropout(self.activation(self.w_1(x))))

class TokenEmbedding(Embedding):
    """A token embedding is a mere synonim of the default torch embedder.
    
    It simply keeps a fixed vocabulary size.
    """
    def __init__(self, vocab_size, embed_size=512):
        super().__init__(vocab_size, embed_size, padding_idx=0)
    

class PositionalEmbedding(Module):
    def __init__(self, d_model, max_len=512):
        """
        Setup positional embeddings
        
        :param d_model: the size of a single embedding
        :param max_len: the maximum number of tokens to embed
        """
        super().__init()
        
        # Compute the positional encodings once in log space
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False
        
        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * (math.log(10000.0) / d_model)).exp()
         
        # I have no clue here
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        return self.pe[:, :x.size(1)]

    
class SegmentEmbedding(Embedding):
    """
    Segment Embedding. It works in a very similar way to token embed but we constrain the size
    of our embedding to 3 (before, current, next sentence?)
    """
    def __init__(self, embed_size=512):
        super().__init__(3, embed_size, padding_idx=0)
        

class BERTEmbedding(Module):
    """
    BERT Embeddings which is made of the following features:
    
    1. Token Embedding: normal embedding matrix
    2. Positional Embedding: additional position information
    3. SegmentEmbedding: Additional sentence segment info (eg. tok_1:1, tok_2:2)
    """
    def __init__(self, vocab_size, embed_size, dropout=0.1):
        """
        :param vocab_size: the size of the vocabulary
        :embed_size: the vector dimensionality of the embeddings
        :dropout: dropout rate
        """
        
        super().__init__()
        self.token = TokenEmbedding(vocab_size=vocab_size, embed_size=embed_size)
        self.position = PositionalEmbedding(embed_size=self.token.embedding_dim)
        self.segment = SegmentEmbedding(embed_size=self.token.embedding_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.embed_size = embed_size
        
    def forward(self, sequence, segment_label):
        x = self.token(sequence) + self.position(sequence) + self.segment(segment_label)
        return self.dropout(x)


class Attention(nn.Module):
    """Attention mechanism in BERT.
    
    Compute "Scaled Dot Product Attention."
    See: https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html
    
    :return return p_attn * value, p_attn (possibly with applied dropout)
    """
    def forward(self, query, key, value, mask=None, dropout: Dropout = None):
        """
        :param query the query matrix
        :param key the key matrix
        :param value the value matrix
        :param mask the mask
        :param dropout an instance of pytor
        """
        
        # apply key to the query to get attention scores
        # Divide by the embedding size to keep the score covariance low
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(query.size(-1))
            
        
        # Be sure to erase out masked values
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # get a probability distribution
        p_attn = F.softmax(scores, dim=-1)
        
        if dropout is not None:
            p_attn = dropout(p_attn)
        
        return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(Module):
    """
    Combine multiple attention heads
    """
    
    def __init__(self, heads, total_embedding_size, dropout=0.1):
        """
        :param heads the number of attention heads. Equivalent to h in the paper
        :param total_embedding_size the size of a token embedding. Must be equal to embedding_per_head * heads,
               where embeddding_per_head. Equivalent to d_k * h in the paper.
        :param dropout: dropout rate
        """
        assert total_embedding_size % heads == 0
        
        self.embedding_per_head = total_embedding_size // heads
        self.heads = heads
        
        # query, key, value
        # Note for myself: ModuleList registers all the modules in autograd
        self.linear_layers = ModuleList([Linear(total_embedding_size, total_embedding_size) for _ in range(3)])
        self.output_linear = Linear(total_embedding_size, total_embedding_size)
        self.attention = Attention()
        
        self.dropout = Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        
        # Note, we only use ONE attention block even though we have different attention heads.
        
        # 1. Perform linear projection in batch from total_embedding_size =>
        #    heads x embedding_per_head
        #
        # Example: query' = self.linear_layers[0](query).view(batch_size, -1, heads, embedding_head)
        # Now, each linear layer has size total_embedding_size x total_embedding_size, while a query has shape
        # batch_size * heads * total_embedding_size ==> (tiled) batch_size * heads * total_embedding_size * total_embedding_size
        query, key, value = [l(x).view(batch_size, -1, self.heads, self.embedding_per_head)
                             for l, x in zip(self.linear_layers, (query, key, value))]
        
        # 2. Apply attention
        x, _ = self.attention(query, key, value, mask=mask, dropout=self.dropout)
    
        # 3. Concat the various attention heads, then project with a FCN
        # Size: batch_size * token_length * heads * self.embedding_per_head
        #
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.heads * self.embedding_per_head)
        
        return self.output_linear(x)

class TransformerBlock(Module):
    """
    Bidirectional (actually a-directional) Encoder = Transformer (self-attention)
    Transformer = MultiHeadedAttention + FeedForward with Sublayer connection
    """
    def __init__(self, embedding_size: int, attn_heads: int, feed_forward_hidden: int,
                 dropout: float):
        """
        :param embedding_size the hidden size of a multiheaded attention layer.
        :param attn_heads the number of heads in the multiheaded layer. Must divide embedding_size.
        :param feed_forward_hidden size of the hidden feed_forward_hidden layer.
        :param dropout the dropout rate
        """
    
        super().__init__()
        self.attention = MultiHeadedAttention(heads=attn_heads, embedding_size=embedding_size)
        self.feed_forward = PositionwiseFeedForward(d_hidden=feed_forward_hidden,
                                                    embedding_size=embedding_size, dropout=dropout)
        self.input_sublayer = SublayerConnection(embedding_size, dropout)
        self.output_sublayer = SublayerConnection(embedding_size, dropout)
        self.dropout = Dropout(dropout)
        
    def forward(self, x, mask):
        x = self.input_sublayer(x, lambda _x: self.attention.forward(_x, _x, _x, mask=mask))
        x = self.output_sublayer(x, self.feed_forward)
        return x.dropout(x)

    
class EntityMemory(Module):
    """
    Entity Memory, as described in the paper
    """
    def __init__(self, embedding_size: int, entity_size: int,
                   entity_embedding_size: int):
        """
        :param embedding_size the size of an embedding. In the EaE paper it is called d_emb, previously as d_k
            (attention_heads * embedding_per_head)
        :param entity_size also known as N in the EaE paper, the maximum number of entities we store
        :param entity_embedding_size also known as d_ent in the EaE paper, the embedding of each entity
        
        """
        self.N = entity_size
        self.d_ent = entity_embedding_size
        self.w_f = Linear(d_ent, 2*embedding_size)
        
    def forward(self, x, entity_spans, num_entities, k=None):
        """
        :param x the (raw) output of the first transformer block. It has a shape:
                B x N x (embed_size)
        :param entity_spans entities and spans of such entities.
                Shape: B x C x 3. Each "row" contains a triple (e_k, s_mi, t_mi)
                where e_k is an (encoded) entity id, s_mi and t_mi are indices.
        :param num_entities the number of found entities for each batch.
        :param k the number of nearest entities to consider when softmax-ing.
                if k = None, all the entities are used.
                In the paper, one should set k for when running the inference
        """
        
        mentions = [entity_spans[:, :mentions_per_batch] for mentions_per_batch in num_entities]
        pass
        

class EaE(Module):
    """
    """
    
    def __init__(self, vocab_size: int, embedding_size: int=768,
                     attn_heads: int = 12, n_layers: int = 12, dropout: float = 0.1):
        """
        :param vocab_size the size of the vocabulary for the token embedder
        :param embedding_size the hidden 
        """
        super().__init__()
        self.embedding_size = embedding_size
        self.n_layers = n_layers
        self.attn_heads = attn_heads
        
        # in the original paper the used embedding size is 512, and the d_ff size is 2048 = 4*512
        # Thus we keep a factor of 4
        self.feed_forward_hidden = 4*embedding_size
        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=embedding_size)
        
        self.transformer_blocks = ModuleList(
            [TransformerBlock(embedding_size, attn_heads, self.feed_forward_hidden, dropout)
            for _ in range(n_layers)])
    
    def forward(self, x, segment_info):
        # x's size: B x T
        # attention masking for padded token (i.e. 0 tokens)
        # x's size after unsqueeze(1): Bx1xT
        # repeat: Bx1xT?
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)
        
        # Get the embeddings for the tokens
        x = self.embedding(x, segment_info)
        
        # Run over multiple transformer blocks
        for transformer in self.transformer_blocks:
            x = transformer.forward(x, mask)
            
        return x
    
